# Проект по SQL

**Актуальность исследования:**
Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке.

**Цель исследования:**
Проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Задания:**

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

# Описание данных

**Таблица books cодержит данные о книгах:**

- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

**Таблица authors содержит данные об авторах:**

- author_id — идентификатор автора;
- author — имя автора.

**Таблица publishers содержит данные об издательствах:**

- publisher_id — идентификатор издательства;
- publisher — название издательства;

**Таблица ratings содержит данные о пользовательских оценках книг:**
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

**Таблица reviews содержит данные о пользовательских обзорах:**

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Сколько книг вышло после 1 января 2000 года**

In [2]:
query_1 = '''SELECT COUNT(*) AS books_after_2000
           FROM books
           WHERE publication_date > '2000-01-01';'''
with engine.connect() as con:
    result_1 = pd.read_sql(sql=text(query_1), con=con)
    display(result_1)

,books_after_2000
0,819


- После 1 января 2000 года всего вышло 819 книг!

**Для каждой книги посчитать количество обзоров и среднюю оценку**



In [3]:
query_2 = '''
SELECT 
    b.title,  
    COUNT(r.review_id) AS review_count,  -- Количество обзоров для каждой книги
    AVG(rt.rating) AS average_rating  -- Средняя оценка для каждой книги
FROM 
    books AS b  -- Основная таблица с книгами
LEFT JOIN 
    reviews AS r ON b.book_id = r.book_id  -- Соединение с таблицей обзоров по идентификатору книги
LEFT JOIN 
    ratings AS rt ON b.book_id = rt.book_id  -- Соединение с таблицей оценок по идентификатору книги
GROUP BY 
    b.book_id, b.title;  -- Группировка по книге и ее названию
'''

# выполняем запрос и выводим результат
with engine.connect() as con:  # открываем соединение с базой данных
    result_2 = pd.read_sql(sql=text(query_2), con=con)  # выполняем SQL-запрос и сохраняем результат в DataFrame
    
    # Вычисляем количество обзоров и среднюю оценку для всех книг
    total_reviews = result_2['review_count'].sum()  # Суммируем все обзоры для всех книг
    average_rating = result_2['average_rating'].mean()  # Находим среднюю оценку по всем книгам
    
    # Выводим результаты
    print(f"Общее количество обзоров: {total_reviews}")  # Выводим общее количество обзоров
    print(f"Средняя оценка всех книг: {average_rating:.2f}")  # Выводим среднюю оценку всех книг с округлением до 2 знаков


Общее количество обзоров: 26167
Средняя оценка всех книг: 3.90


- Обзоров на книги 26167, а вот их средняя оценка(рейтинг) 3.9 не самый хороший показатель!

**Определить издательство, которое выпустило наибольшее число книг толще 50 страниц**

In [4]:
query_3 = '''
SELECT 
    p.publisher,  -- название издательства
    COUNT(b.book_id) AS book_count  -- количество книг
FROM 
    books AS b
JOIN 
    publishers AS p ON b.publisher_id = p.publisher_id  -- соединяем книги с издательствами
WHERE 
    b.num_pages > 50  -- фильтруем книги, толще 50 страниц
GROUP BY 
    p.publisher  -- группируем по издательствам
ORDER BY 
    book_count DESC  -- сортируем по количеству книг
LIMIT 1;  -- выводим только издательство с наибольшим количеством книг
'''

# выполняем запрос и выводим результат
with engine.connect() as con:  # открываем соединение
    result_3 = pd.read_sql(sql=text(query_3), con=con)  # выполняем запрос
    
    # Выводим издательство с наибольшим количеством книг
    print(f"Издательство с наибольшим количеством книг (толще 50 страниц): {result_3['publisher'][0]}")
    print(f"Количество книг: {result_3['book_count'][0]}")


Издательство с наибольшим количеством книг (толще 50 страниц): Penguin Books
Количество книг: 42


- Издательство Penguin Books больше предпочитает выпускать книги с долгой историей, а переводить бумагу на "брошюру" не любит!

In [5]:
query_4 = '''
SELECT 
    a.author,  
    AVG(rt.rating) AS average_rating  -- средняя оценка автора
FROM 
    books AS b
JOIN 
    authors AS a ON b.author_id = a.author_id  -- соединяем книги с авторами
JOIN 
    ratings AS rt ON b.book_id = rt.book_id  -- соединяем книги с оценками
GROUP BY 
    a.author  -- группируем по авторам
HAVING 
    COUNT(rt.rating) >= 50  -- учитываем только книги с 50 и более оценками
ORDER BY 
    average_rating DESC  -- сортируем по средней оценке в убывающем порядке
LIMIT 1;  -- выводим только автора с самой высокой средней оценкой
'''

# выполняем запрос и выводим результат
with engine.connect() as con:  # открываем соединение
    result_4 = pd.read_sql(sql=text(query_4), con=con)  # выполняем запрос
    
    # Выводим автора с самой высокой средней оценкой
    print(f"Автор с самой высокой средней оценкой книг: {result_4['author'][0]}")
    print(f"Средняя оценка: {result_4['average_rating'][0]:.2f}")


Автор с самой высокой средней оценкой книг: Diana Gabaldon
Средняя оценка: 4.30


- Творчество автора Diana Gabaldon со средней оценкой в 4.3 оценивают выше всех других!

In [7]:
query_5 = '''
SELECT 
    AVG(review_count) AS average_reviews_per_user  -- Среднее количество обзоров для пользователей с более чем 48 оценками
FROM (
    SELECT 
        rt.username,  -- Имя пользователя из таблицы ratings
        COUNT(rt.rating) AS rating_count,  -- Количество оценок, оставленных пользователем
        COUNT(rv.review_id) AS review_count  -- Количество обзоров, оставленных пользователем
    FROM 
        ratings AS rt  -- Таблица оценок
    LEFT JOIN 
        reviews AS rv ON rt.book_id = rv.book_id  -- Соединение с таблицей обзоров
    GROUP BY 
        rt.username  -- Группировка по пользователю (используем rt для ratings)
    HAVING 
        COUNT(rt.rating) > 48  -- Учитываем только пользователей с более чем 48 оценками
) AS user_reviews;  -- Вложенный запрос для получения данных по пользователям
'''

# выполняем запрос и выводим результат
with engine.connect() as con:  # открываем соединение с базой данных
    result_5 = pd.read_sql(sql=text(query_5), con=con)  # выполняем SQL-запрос и сохраняем результат в DataFrame
    
    # Выводим результат
    print(f"Среднее количество обзоров от пользователей с более чем 48 оценками: {result_5['average_reviews_per_user'][0]:.2f}")


Среднее количество обзоров от пользователей с более чем 48 оценками: 163.54


Самые активные читатели с 48-ю отзывами и более в среднем делают 163-164 обзоров на книги!